# ---- PREPARING INFORMATION FOR FASTAPI ----

This notebook contains the necessary transformations on the information to be processed by the functions developed for the API. The data sources are the files: items.csv, reviews.csv, games.csv, and reviews_sa.csv

In [15]:
import pandas as pd                                         # ---> Libraries to be used
import ast

In [16]:
df_items = pd.read_csv('./Datasets/processing/items.csv')   # ---> Reading of csv files previously processed with ETL techniques
df_games = pd.read_csv('./Datasets/processing/games.csv')             
df_reviews = pd.read_csv('./Datasets/processing/reviews_sa.csv')

### 1. Data preparation to answer 'Endpoint1 = PlayTimeGenre'

In [17]:
# ---> An Inner Merge of the DataFrames: df_items, df_games, on the column 'item_id' is performed
df_items_games = pd.merge(df_items, df_games[['item_id', 'released_year', 'genres']], on='item_id', how='inner')
df_items_games.head()

,item_id,user_id,hours_game,released_year,genres
0,10,76561197970982479,0.10,2000,Action
1,10,doctr,1.55,2000,Action
2,10,corrupted_soul,1.80,2000,Action
3,10,WeiEDKrSat,5.47,2000,Action
4,10,death-hunter,104.58,2000,Action


In [18]:
# ---> It is grouped by genre and year of release, and the hours of play are added up (rounded to 2 decimal places)
df_items_games = df_items_games.groupby(['released_year', 'genres'])['hours_game'].sum().reset_index()
df_items_games['hours_game'] = round(df_items_games['hours_game'], 2)
df_items_games.head()

,released_year,genres,hours_game
0,1983,Action,57.92
1,1983,Adventure,57.92
2,1983,Casual,57.92
3,1984,Action,6.40
4,1984,Adventure,6.40


In [19]:
# ---> Calculation to rank the genres with the most hours played per year 
df_genre_top_hoursgame = df_items_games.loc[df_items_games.groupby(['released_year', 'genres'])['hours_game'].idxmax()]

In [20]:
print(df_genre_top_hoursgame)

     released_year        genres  hours_game
0             1983        Action       57.92
1             1983     Adventure       57.92
2             1983        Casual       57.92
3             1984        Action        6.40
4             1984     Adventure        6.40
..             ...           ...         ...
335           2018     Adventure        0.35
336           2018  Early Access      102.12
337           2018         Indie       38.78
338           2018           RPG       63.69
339           2018    Simulation       36.71

[340 rows x 3 columns]


In [21]:
# ---> Routine to order the DataFrame: 'df_genre_top_hoursgame'
genre_top_hoursgame = df_genre_top_hoursgame.sort_values(by=['genres', 'hours_game','released_year'], ascending=[True, False, True])
print(df_genre_top_hoursgame)

     released_year        genres  hours_game
0             1983        Action       57.92
1             1983     Adventure       57.92
2             1983        Casual       57.92
3             1984        Action        6.40
4             1984     Adventure        6.40
..             ...           ...         ...
335           2018     Adventure        0.35
336           2018  Early Access      102.12
337           2018         Indie       38.78
338           2018           RPG       63.69
339           2018    Simulation       36.71

[340 rows x 3 columns]


In [22]:
# ---> Conversion of 'genres' column to uppercaps
genre_top_hoursgame['genres'] = genre_top_hoursgame['genres'].str.upper()

In [23]:
print(genre_top_hoursgame)

     released_year          genres   hours_game
216           2012          ACTION  18093686.49
234           2013          ACTION   6910695.30
205           2011          ACTION   5526109.74
275           2015          ACTION   4364111.78
193           2010          ACTION   3001469.08
..             ...             ...          ...
135           2005  WEB PUBLISHING      1564.19
274           2014  WEB PUBLISHING       560.80
165           2007  WEB PUBLISHING       184.80
333           2017  WEB PUBLISHING       156.37
316           2016  WEB PUBLISHING         2.27

[340 rows x 3 columns]


In [24]:
genre_top_hoursgame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 340 entries, 216 to 316
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   released_year  340 non-null    int64  
 1   genres         340 non-null    object 
 2   hours_game     340 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 10.6+ KB


In [25]:
# ---> Upload results to a CSV file
df_genre_top_hoursgame.to_csv('./Datasets/api/playtimegenre.csv', encoding='utf-8', index=False)

### 2. Data preparation to answer 'Endpoint2 = UserForGenre'

In [26]:
# ---> A Merge of the DataFrames: df_items, df_games, on the column 'item_id' is performed
df_items_games2 = pd.merge(df_items,df_games,left_on='item_id', right_on='item_id', how='right')
df_items_games2.head()

,item_id,user_id,hours_game,genres,app_name,specs,developer,released_year
0,761140,NaN,NaN,Action,Lost Summoner Kitty,['Single-player'],Kotoshiro,2018
1,761140,NaN,NaN,Casual,Lost Summoner Kitty,['Single-player'],Kotoshiro,2018
2,761140,NaN,NaN,Indie,Lost Summoner Kitty,['Single-player'],Kotoshiro,2018
3,761140,NaN,NaN,Simulation,Lost Summoner Kitty,['Single-player'],Kotoshiro,2018
4,761140,NaN,NaN,Strategy,Lost Summoner Kitty,['Single-player'],Kotoshiro,2018


In [27]:
df_items_games2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6862422 entries, 0 to 6862421
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   item_id        int64  
 1   user_id        object 
 2   hours_game     float64
 3   genres         object 
 4   app_name       object 
 5   specs          object 
 6   developer      object 
 7   released_year  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 418.8+ MB


In [28]:
df_items_games2.shape

(6862422, 8)

In [29]:
# ---> Removal of non-relevant information
df_items_games2.drop(['item_id'], axis=1, inplace=True)
df_items_games2.dropna(inplace=True)
df_items_games2.shape

(6796445, 7)

In [30]:
# ---> Conversion from Float to Integer
df_items_games2['hours_game'] = df_items_games2['hours_game'].astype('int64')

In [31]:
df_items_games2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6796445 entries, 75 to 6862388
Data columns (total 7 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   user_id        object
 1   hours_game     int64 
 2   genres         object
 3   app_name       object
 4   specs          object
 5   developer      object
 6   released_year  int64 
dtypes: int64(2), object(5)
memory usage: 414.8+ MB


In [32]:
# ---> Calculation to rank users with the most hours played by gender and year
df_topuser_genrehours = df_items_games2.loc[df_items_games2.groupby(['genres', 'released_year'])['hours_game'].idxmax()]
df_topuser_genrehours.head()

,user_id,hours_game,genres,app_name,specs,developer,released_year
1154691,76561197966936422,5,Action,Dragon's Lair,"['Single-player', 'Steam Achievements', 'Parti...",Digital Leisure Inc.,1983
1194878,76561198064710856,1,Action,Space Ace,"['Single-player', 'Steam Achievements', 'Parti...",Digital Leisure Inc.,1984
4243070,DownSyndromeKid,11,Action,Gold Rush! Classic,"['Single-player', 'Steam Trading Cards']","The Software Farm,Sunlight Games,Sierra Entert...",1988
4298369,starfleetbrat,8,Action,Tex Murphy: Mean Streets,['Single-player'],Access Software,1989
6741393,76561198041356854,23,Action,Commander Keen,"['Single-player', 'Steam Cloud']",id Software,1990


In [33]:
# ---> filter out columns that are not required
df_topuser_genrehours = df_topuser_genrehours[['genres', 'released_year', 'user_id', 'hours_game']]
df_topuser_genrehours.head()

,genres,released_year,user_id,hours_game
1154691,Action,1983,76561197966936422,5
1194878,Action,1984,76561198064710856,1
4243070,Action,1988,DownSyndromeKid,11
4298369,Action,1989,starfleetbrat,8
6741393,Action,1990,76561198041356854,23


In [34]:
# ---> Conversion of 'genres' column to uppercaps
df_topuser_genrehours['genres'] = df_topuser_genrehours['genres'].str.upper()

In [35]:
print(df_topuser_genrehours)

                 genres  released_year            user_id  hours_game
1154691          ACTION           1983  76561197966936422           5
1194878          ACTION           1984  76561198064710856           1
4243070          ACTION           1988    DownSyndromeKid          11
4298369          ACTION           1989      starfleetbrat           8
6741393          ACTION           1990  76561198041356854          23
...                 ...            ...                ...         ...
1384613  WEB PUBLISHING           2013  76561198059330972        1090
4525395  WEB PUBLISHING           2014   Dark_samaCH_PONY          94
3485777  WEB PUBLISHING           2015            Xyphien        1183
2605113  WEB PUBLISHING           2016  76561198013000952           0
1147485  WEB PUBLISHING           2017  76561198045638432          31

[322 rows x 4 columns]


In [36]:
# ---> Calculation to obtain the hours played per year for the selected user
df_topuser_genrehours2 = df_topuser_genrehours.groupby(['genres', 'released_year', 'user_id'])['hours_game'].sum().reset_index()
df_topuser_genrehours2.head()

,genres,released_year,user_id,hours_game
0,ACTION,1983,76561197966936422,5
1,ACTION,1984,76561198064710856,1
2,ACTION,1988,DownSyndromeKid,11
3,ACTION,1989,starfleetbrat,8
4,ACTION,1990,76561198041356854,23


In [37]:
# ---> Upload results to a CSV file
df_topuser_genrehours2.to_csv('./Datasets/api/userforgenre.csv', encoding='utf-8', index=False)

### 3. Data preparation to answer 'Endpoint3 = UsersRecommend'

In [38]:
# ---> A Merge of the DataFrames: df_reviews, df_games, on the column 'item_id' is performed
df_reviews_games = pd.merge(df_reviews, df_games, on='item_id', how='left')
df_reviews_games.head()

,item_id,recommend,posted_year,sentiment,genres,app_name,specs,developer,released_year
0,1250,True,2011,2,Action,Killing Floor,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",Tripwire Interactive,2009.0
1,22200,True,2011,2,Action,Zeno Clash,"['Single-player', 'Steam Achievements', 'Steam...",ACE Team,2009.0
2,22200,True,2011,2,Indie,Zeno Clash,"['Single-player', 'Steam Achievements', 'Steam...",ACE Team,2009.0
3,43110,True,2011,2,NaN,NaN,NaN,NaN,NaN
4,251610,True,2014,2,NaN,NaN,NaN,NaN,NaN


In [39]:
df_reviews_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109403 entries, 0 to 109402
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   item_id        109403 non-null  int64  
 1   recommend      109403 non-null  bool   
 2   posted_year    109403 non-null  int64  
 3   sentiment      109403 non-null  int64  
 4   genres         101865 non-null  object 
 5   app_name       101865 non-null  object 
 6   specs          101821 non-null  object 
 7   developer      101865 non-null  object 
 8   released_year  101865 non-null  float64
dtypes: bool(1), float64(1), int64(3), object(4)
memory usage: 6.8+ MB


In [40]:
# ---> Filter a new DataFrame by columns: 'recommend' and 'sentiment' (neutral and positive scores)
df_conditions = df_reviews_games.loc[(df_reviews_games['recommend'] == True) & (df_reviews_games['sentiment'].isin([1, 2])), ['released_year', 'app_name']]
print(df_conditions)

        released_year                          app_name
0              2009.0                     Killing Floor
1              2009.0                        Zeno Clash
2              2009.0                        Zeno Clash
3                 NaN                               NaN
4                 NaN                               NaN
...               ...                               ...
109397         2012.0  Counter-Strike: Global Offensive
109398         2012.0  Counter-Strike: Global Offensive
109399         2003.0                           Enclave
109401         2012.0  Counter-Strike: Global Offensive
109402            NaN                               NaN

[85590 rows x 2 columns]


In [41]:
# ---> Grouping by columns: 'released_year' and 'app_name' and counting occurrences
df_conditions1 = df_conditions.groupby(['released_year', 'app_name']).size().reset_index(name='count')

In [42]:
# ---> Sort by columns: 'released_year' and 'count' in descending order
df_conditions1 = df_conditions1.sort_values(by=['released_year', 'count'], ascending=[False, False])
df_conditions1.head()

,released_year,app_name,count
2075,2017.0,Unturned,3630
2070,2017.0,Robocraft,1194
2050,2017.0,ARK: Survival Evolved,665
2058,2017.0,Dungeon Defenders II,140
2073,2017.0,The Long Dark,104


In [43]:
df_conditions1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2077 entries, 2075 to 0
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   released_year  2077 non-null   float64
 1   app_name       2077 non-null   object 
 2   count          2077 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 64.9+ KB


In [44]:
# ---> Conversion from Float to Integer
df_conditions1['released_year'] = df_conditions1['released_year'].astype('int64')

In [45]:
df_conditions1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2077 entries, 2075 to 0
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   released_year  2077 non-null   int64 
 1   app_name       2077 non-null   object
 2   count          2077 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 64.9+ KB


In [46]:
# ---> Top 3 of game recommendations by year
df_top_year = df_conditions1.groupby('released_year').head(3)
print(df_top_year)

      released_year               app_name  count
2075           2017               Unturned   3630
2070           2017              Robocraft   1194
2050           2017  ARK: Survival Evolved    665
2040           2016              Starbound   1345
2017           2016      Heroes & Generals    460
...             ...                    ...    ...
5              1992            Word Rescue      2
3              1991          Crystal Caves      1
1              1990         Commander Keen      3
2              1990                  LOOM™      1
0              1989   Sword of the Samurai      2

[80 rows x 3 columns]


In [47]:
# ---> Upload results to a CSV file
df_top_year.to_csv('./Datasets/api/usersrecommend.csv', encoding='utf-8', index=False)

### 4. Data preparation to answer 'Endpoint4 = UsersWorstDeveloper'

In [48]:
# ---> A Merge of the DataFrames: df_reviews, df_games, on the column 'item_id' is performed
df_reviews_games4 = pd.merge(df_reviews, df_games, on='item_id', how='left')
df_reviews_games4.head()

,item_id,recommend,posted_year,sentiment,genres,app_name,specs,developer,released_year
0,1250,True,2011,2,Action,Killing Floor,"['Single-player', 'Multi-player', 'Co-op', 'Cr...",Tripwire Interactive,2009.0
1,22200,True,2011,2,Action,Zeno Clash,"['Single-player', 'Steam Achievements', 'Steam...",ACE Team,2009.0
2,22200,True,2011,2,Indie,Zeno Clash,"['Single-player', 'Steam Achievements', 'Steam...",ACE Team,2009.0
3,43110,True,2011,2,NaN,NaN,NaN,NaN,NaN
4,251610,True,2014,2,NaN,NaN,NaN,NaN,NaN


In [49]:
# ---> Filter a new DataFrame by columns: 'recommend' and 'sentiment' (negative score)
df_reviews_games41 = df_reviews_games4.loc[(df_reviews_games4['recommend'] == False) & (df_reviews_games4['sentiment'] == 0), ['released_year', 'developer']]

In [50]:
df_reviews_games41.head()

,released_year,developer
112,NaN,NaN
249,2015.0,Code}{atch
250,2015.0,Code}{atch
251,2015.0,Code}{atch
252,2015.0,Code}{atch


In [51]:
df_reviews_games41.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5074 entries, 112 to 109367
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   released_year  4575 non-null   float64
 1   developer      4575 non-null   object 
dtypes: float64(1), object(1)
memory usage: 118.9+ KB


In [52]:
# ---> Calculation of the occurrences of each developer per year
df_reviews_games42 = df_reviews_games41.groupby(['released_year', 'developer']).size().reset_index(name='count')
df_reviews_games42.head()

,released_year,developer,count
0,1990.0,id Software,1
1,1998.0,"Jackbox Games, Inc.",2
2,1999.0,Chris Sawyer Productions,2
3,1999.0,Valve,1
4,2000.0,Ion Storm,1


In [53]:
# ---> Conversion from Float to Integer
df_reviews_games42['released_year'] = df_reviews_games42['released_year'].astype('int64')

In [54]:
# ---> Sort by columns: 'released_year' and 'count' in descending order
df_reviews_games42 = df_reviews_games42.sort_values(by=['released_year', 'count'], ascending=[False, False])
df_reviews_games42.head()

,released_year,developer,count
583,2017,Smartly Dressed Games,190
580,2017,Freejam,144
584,2017,"Studio Wildcard,Instinct Games,Efecto Studios,...",95
581,2017,Hoobalugalar_X,10
586,2017,Trendy Entertainment,10


In [55]:
# ---> Top 3 of worst developers by year
df_top_worstdev = df_reviews_games42.groupby('released_year').head(3)    

In [56]:
df_top_worstdev.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53 entries, 583 to 0
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   released_year  53 non-null     int64 
 1   developer      53 non-null     object
 2   count          53 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.7+ KB


In [57]:
# ---> Upload results to a CSV file
df_top_worstdev.to_csv('./Datasets/api/worstdeveloper.csv', encoding='utf-8', index=False)

### 5. Data preparation to answer 'Endpoint5 = Sentiment_analysis'

In [58]:
# ---> A Merge of the DataFrames: df_reviews, df_games, on the column 'item_id' is performed
df_reviews_games5 = pd.merge(df_games, df_reviews, on='item_id', how='inner')
df_reviews_games5.head()

,genres,app_name,specs,item_id,developer,released_year,recommend,posted_year,sentiment
0,Action,Half-Life,"['Single-player', 'Multi-player', 'Valve Anti-...",70,Valve,1998,True,2015,2
1,Action,Half-Life,"['Single-player', 'Multi-player', 'Valve Anti-...",70,Valve,1998,True,2011,0
2,Action,Half-Life,"['Single-player', 'Multi-player', 'Valve Anti-...",70,Valve,1998,True,2014,2
3,Action,Half-Life,"['Single-player', 'Multi-player', 'Valve Anti-...",70,Valve,1998,True,2013,2
4,Action,Half-Life,"['Single-player', 'Multi-player', 'Valve Anti-...",70,Valve,1998,True,2013,2


In [59]:
# ---> Group by ''developer' and 'sentiment' columns, and count occurrences
df_reviews_games51 = df_reviews_games5.groupby(['developer', 'sentiment']).size().reset_index(name='count')

In [60]:
df_reviews_games51.head()

,developer,sentiment,count
0,07th Expansion,0,1
1,07th Expansion,1,1
2,"10th Art Studio,Adventure Productions",0,2
3,"10th Art Studio,Adventure Productions",2,2
4,11 bit studios,0,63


In [61]:
# ---> Pivot the DataFrame to have sentiments (score) as columns
df_reviews_games51 = df_reviews_games51.pivot(index='developer', columns='sentiment', values='count').reset_index()

In [62]:
df_reviews_games51.head()

sentiment,developer,0,1,2
0,07th Expansion,1.0,1.0,NaN
1,"10th Art Studio,Adventure Productions",2.0,NaN,2.0
2,11 bit studios,63.0,15.0,60.0
3,14° East,1.0,NaN,1.0
4,17-BIT,NaN,NaN,4.0


In [63]:
df_reviews_games51.columns = ['developer', 'Negative', 'Neutral', 'Positive']

In [64]:
print(df_reviews_games51.isna().sum())

developer      0
Negative     794
Neutral      896
Positive     208
dtype: int64


In [65]:
# ---> Fill NaN with 0
df_sentiment = df_reviews_games51.fillna(0)

In [66]:
# ---> Conversion from Float to Integer
df_sentiment[['Negative', 'Neutral', 'Positive']] = df_sentiment[['Negative', 'Neutral', 'Positive']].astype('int64')

In [67]:
df_sentiment.head()

,developer,Negative,Neutral,Positive
0,07th Expansion,1,1,0
1,"10th Art Studio,Adventure Productions",2,0,2
2,11 bit studios,63,15,60
3,14° East,1,0,1
4,17-BIT,0,0,4


In [68]:
df_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1673 entries, 0 to 1672
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   developer  1673 non-null   object
 1   Negative   1673 non-null   int64 
 2   Neutral    1673 non-null   int64 
 3   Positive   1673 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 52.4+ KB


In [69]:
# ---> Upload results to a CSV file
df_sentiment.to_csv('./Datasets/api/sentiment_analysis.csv', index=False)